# Building an auto-complete feature for citizen science checklist submissions 

In this project we try to create a recommender system that can be used to impute the missing values in the ebirds data set.we work with the training data set which has complete and accurate values and use  that to train our algorithim which then is used to recommend possible bird values in the test data set where we have soft zero values.

**importing and loading necessary libraries**

The tqdm library is used to check the progress of our algorithim as it runs i.e states how many columns are remaining to iterate over, time the code takes to run etc 

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm

*Reading in the data files*

In [3]:
training_set = pd.read_csv('training_set[1].csv')
test_set = pd.read_csv('test_set[1].csv')
sample_submission = pd.read_csv('sample_submission[1].csv')

In [4]:
# using .head() function to check the content of the test set
test_set.head()

,Unnamed: 0,test1,test2,test3,test4,test6,test7,test8,test9,test10,...,test6348,test6349,test6350,test6351,test6352,test6353,test6354,test6355,test6356,test6357
0,Apapane,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Hawaii_Elepaio,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Kalij_Pheasant,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Northern_Cardinal,0,0,2,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
4,Omao,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
training_set.head()

,Unnamed: 0,train1,train2,train3,train4,train5,train6,train7,train8,train9,...,train14823,train14824,train14825,train14826,train14827,train14828,train14829,train14830,train14831,train14832
0,Apapane,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Hawaii_Elepaio,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Kalij_Pheasant,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Northern_Cardinal,1,0,1,4,4,0,2,2,0,...,1,2,1,3,0,0,0,0,1,0
4,Omao,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# removing the first column from the training_set data and test_set data frame inorder to aid in preprocessing
# i.e so that we can have only numerical values ,the first column contains bird names 
    
columns2 = training_set.columns[1:]
training_set2 = training_set[columns2]

columns3 = test_set.columns[1:]
test_set2 = test_set[columns3]

## Preprocessing data

first both data sets are scaled to have values between zero and one and we created new scaled data sets scaled_training1 and scaled_test1

In [7]:
# function to scale data so that all values in a column add up to 1
def scale(df):
    """Normalize all columns in a DataFrame so that their values add up to 1"""
    for column in df.columns:
        column_sum = df[column].sum()
        df[column] = df[column] / column_sum
    return df

In [8]:
# Applying the function to the two data sets
scaled_training = scale(training_set2)
scaled_test = scale(test_set2)

scaled_training1= scaled_training.to_numpy()
scaled_test1 = scaled_test.to_numpy()



C:\Users\MAURINE\AppData\Local\Temp\ipykernel_9304\3945039910.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column] / column_sum


##  functions created for the project

A number of functions were created to help in runing the algorithim to ensure that it does what it is expected to do.

In [9]:
# function that calculates the euclidean distance between two data points
# these calculated distances will be used in determining the nearest neighbors by selecting the closest samples

def euclidean_distance(x,y):
    distance = np.sqrt(np.sum((x-y)**2))
    return distance
    


The cosine_distance function was also created though using it in our knn algorithm produced poor scores so it was not used in producing the final score.It calculates the cosine distance between two arrays x and y

In [10]:
def cosine_distance(x, y):
    # Ensure that x and y are numpy arrays
    x = np.array(x)
    y = np.array(y)
   
    # Calculate the dot product
    dot_product = np.dot(x, y)
    
    # Calculate the magnitudes
    x_magnitude = np.linalg.norm(x)
    y_magnitude = np.linalg.norm(y)
    
    # Calculate the cosine distance
    cosine_dist = 1-(dot_product / (x_magnitude * y_magnitude))
    
    return cosine_dist

In an attempt to only calculate the distances between the zero values in the test data and their corresponding values in the training data set the process_mask function was created but the scores were not too good hence was not utilized in getting the final score.
The function calculates the euclidean distances between this values in the two samples(one test sample and all training samples) ,sorts and gets the k nearest samples, replaces their values with 1,gets their indices in the training data set and forms a new matrix Z with these nearest samples,calculates the row means, if the corresponding value in the test column is not a zero replaces the value in the training sample row means with a zero, then replaces the top 5 row means with 1 and the rest with zero and returns that matrix. 

Where A is the training data set, B is one test sample, and K is the choosen value of neighbors.

In [12]:
# function that finds the k-nearest neighbors by calculating the distances only between zero values in the test data. 

def process_mask(A,B,K):
    m, n = A.shape
    mask = B == 0
    A_masked = A[mask,:]
    B_masked = B[mask]
    training_nn = np.zeros(n)
    for i in range(A_masked.shape[1]):
        column_i = A_masked[:, i]
        training_nn[i] = euclidean_distance(column_i, B_masked)
    training_nn[np.argsort(training_nn)[:K]] = 1
    Z = A[:, training_nn == 1]
    recommend = Z.mean(axis=1)
    for i in range(recommend.shape[0]):
        if B[i] != 0:
            recommend[i] = 0
    recommend[np.argsort(recommend)[-5:]] = 1
    recommend[np.argsort(recommend)[:-5]] = 0
    return recommend
    
    

The nearest _neighbors function does the same thing as the process_mask function above the only
difference being that it calculates the distances between all the values and not only the zero values.This is the function that produced the best results and hence was used to produce the sample submission with a good score.

In [15]:
# function that finds the k-nearest neighbors by taking in the training data set,one test sample and k as arguments
# then calculates the distances between all the values and not only  zero
# then does all the steps as in the process_mask function above to return the matrix with recommendations.

def nearest_neighbors(A,B,K):
    m, n = A.shape
    training_nn = np.zeros(n)
    for i in range(n):
        training_nn[i] = euclidean_distance(A[:, i], B)
    training_nn[np.argsort(training_nn)[:K]] = 1
    Z = A[:, training_nn == 1]
    recommend = Z.mean(axis=1)
    for i in range(recommend.shape[0]):
        if B[i] != 0:
            recommend[i] = 0
    recommend[np.argsort(recommend)[-5:]] = 1
    recommend[np.argsort(recommend)[:-5]] = 0
    return recommend

The **recommender function** then iterates through all the the test samples and uses the nearest_neighbors function to produce recommendation arrays which are the appended to an empty numpy array  called recommendations.The tqdm software is used here  to check the progress of the running code.

In [16]:
# The reommender function takes in the scaled training data set, scaled test data set and k value as arguments.
# It then returns a numpy array with all the recommendations for all test samples.
def recommender(A,C,K):
    recommendations = np.empty((C.shape[0], 0))
    for col in tqdm(C.T):
        recommend = nearest_neighbors(A,col,K)
        recommendations = np.append(recommendations, recommend.reshape(-1, 1), axis=1)
    return recommendations

### Running the function on our data sets to come up with recommendations

In [18]:
# Here we see the recommender function being implemented on the scaled training data set ,and scaled test data.
# we can also see the tqdm software in action as the code runs.
recommendations = recommender(scaled_training1,scaled_test1,75)

100%|██████████████████████████████████████████████████████████████████████████████| 6009/6009 [37:07<00:00,  2.70it/s]


### checking the validity of our results and preprocessing the recommendation to have it in the correct format for submission.

In the code chunk below we calculate the sum of the values in the columns in our recommendations numpy array to confirm that there are only five recommendations per test sample

In [19]:
# using np.sum to calculate te sum in the columns i.e axis 0
sum_of_columns = np.sum(recommendations, axis=0)

print(sum_of_columns)

[5. 5. 5. ... 5. 5. 5.]


In the code below we convert the array into a pandas data frame to help us change its format from wide to long later on using the pandas melt function.We also define the type of the data frame as int and set the index just to ensure we have all the birds from the checklist.

In [20]:
df = pd.DataFrame(recommendations)
df = df.astype(int)
df = df.set_index(test_set.loc[:,'Unnamed: 0'])

In [27]:
# printing out the data frame to see how it looks
df

,0,1,2,3,4,5,6,7,8,9,...,5999,6000,6001,6002,6003,6004,6005,6006,6007,6008
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
Apapane,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Hawaii_Elepaio,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Kalij_Pheasant,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Northern_Cardinal,0,0,0,1,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
Omao,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Greater_White_fronted_Goose,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Mourning_Dove,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Pectoral_Sandpiper,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


We then convert the data frame from the wide format to the long format using pd.melt function as shown below

In [21]:
# converting to tall format from wide format
df_long = pd.melt(df, value_vars=df.columns,value_name='value')
df_long

,variable,value
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
510760,6008,0
510761,6008,0
510762,6008,0
510763,6008,0


In [22]:
# dropping the variable column
df_long = df_long.drop(['variable'], axis=1)
df_long.head()

,value
0,0
1,0
2,0
3,0
4,0


we then replace the values in the sample_submission file with the values in our data frame in preparation for submission

In [24]:
# replacing the values in the Expected column of the sample_submission with our values
sample_submission['Expected'] = df_long['value'].values

In [31]:
sample_submission

,Unnamed: 0,Id,Expected
0,1,row_1,0
1,2,row_2,0
2,3,row_3,0
3,4,row_4,0
4,5,row_5,0
...,...,...,...
510760,510761,row_510761,0
510761,510762,row_510762,0
510762,510763,row_510763,0
510763,510764,row_510764,0


In [25]:
# removing the unwanted columns so that we only remain with two columns for submission
sample_submissionk = sample_submission.columns[1:]
sample_sub_final = sample_submission[sample_submissionk]
sample_sub_final

,Id,Expected
0,row_1,0
1,row_2,0
2,row_3,0
3,row_4,0
4,row_5,0
...,...,...
510760,row_510761,0
510761,row_510762,0
510762,row_510763,0
510763,row_510764,0


### Converting the final data frame to csv for submission

In [27]:
 sample_sub_final.to_csv("final_submission21.csv", index = False)